In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
import random
import numpy as np
from tqdm import tqdm
import argparse
import wandb
from os.path import splitext
from os import listdir
import numpy as np
import os
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image
from torchvision.transforms import functional as TF
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import imageio.v2 as imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision import models
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import glob
import wandb
import random
import numpy as np
from model import *
from utils import *
from dataset import *

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
run = wandb.init()

wandb: Currently logged in as: stmmc (tousi-team). Use `wandb login --relogin` to force relogin


In [5]:
dem_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/New_Data/dem'
so_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/New_Data/so'
rgb_dir = '/home/macula/SMATousi/Gullies/ground_truth/google_api/training_process/DEM2SO/New_Data/rgb'
pretrained_model_path = '/home/macula/SMATousi/cluster/docker-images/dem2so_more_data/pre_models/B3_rn50_moco_0099_ckpt.pth'



In [6]:
transform = RGB_RasterTransform()
    
dataset = RGB_RasterTilesDataset(dem_dir=dem_dir, so_dir=so_dir, rgb_dir=rgb_dir, transform=transform)

In [7]:
batch_size = 1
learning_rate = 0.0001
epochs = 1
number_of_workers = 0
image_size = 128
val_percent = 0.1

n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=number_of_workers, pin_memory=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=number_of_workers, pin_memory=True, drop_last=True)

    

In [8]:
model = RGB_DEM_to_SO(resnet_output_size=(8, 8), 
                            fusion_output_size=(128, 128), 
                            model_choice = "Unet_1", 
                            resnet_saved_model_path=pretrained_model_path,
                            dropout_rate=0.5)

/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
from torch.optim import Adam
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

In [12]:
artifact = run.use_artifact('tousi-team/RGB_DEM_2_SO/model_epoch_200:v6', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact model_epoch_200:v6, 144.43MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:8.6


In [13]:
model.load_state_dict(torch.load('./artifacts/model_epoch_200:v6/model_epoch_200.pth'))

<All keys matched successfully>

In [57]:
def mIOU(label, pred, num_classes=9):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    iou_list = list()
    present_iou_list = list()

    pred = pred.view(-1)
    label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        pred_inds = (pred == sem_class)
        target_inds = (label == sem_class)
        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')
        else: 
            intersection_now = (pred_inds[target_inds]).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)
            # print(iou_now)
        iou_list.append(iou_now)
    return present_iou_list, np.mean(present_iou_list)

import torch
import numpy as np

def mIOU_with_tolerance(label, pred, num_classes=9):
    pred = torch.softmax(pred, dim=1)
    pred = torch.argmax(pred, dim=1)

    # Flatten label and prediction tensors
    pred = pred.view(-1)
    label = label.view(-1)

    iou_list = []
    present_iou_list = []

    for sem_class in range(num_classes):
        # Adjusting for tolerance: consider predictions that are one level above or below the target class
        pred_inds = ((pred == sem_class) | (pred == sem_class - 1) & (pred > 0) | (pred == sem_class + 1) & (pred < num_classes - 1))
        target_inds = ((label == sem_class) | (label == sem_class - 1) & (label > 0) | (label == sem_class + 1) & (label < num_classes - 1))

        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')  # Class not present in the label
        else:
            intersection_now = (pred_inds & target_inds).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)

        iou_list.append(iou_now)

    # Compute the mean of the present IoU scores, ignoring NaNs
    valid_iou_scores = [iou for iou in present_iou_list if not np.isnan(iou)]
    return present_iou_list, valid_iou_scores, np.mean(valid_iou_scores) if valid_iou_scores else float('nan')

# Note: You'll need to replace 'F.softmax' and 'torch.argmax' with the appropriate calls to your prediction tensor,
# and ensure your label and pred tensors are in the correct shape and format.

def normalize_and_discretize_for_mIoU(label, pred, num_classes=9):
    # Ensure pred and label are torch tensors
    pred = torch.tensor(pred, dtype=torch.float32)
    label = torch.tensor(label, dtype=torch.float32)
    
    # Normalize by dividing by the maximum value in each tensor
    pred_argmax = torch.argmax(pred, dim=1)
    pred_max = pred_argmax.max() if pred_argmax.max() > 0 else 1  # Avoid division by zero
    label_max = label.max() if label.max() > 0 else 1
    pred_normalized = pred_argmax / pred_max
    label_normalized = label / label_max
    
    print(pred_normalized.shape)
    
    print(label_normalized.shape)

    # Discretize the normalized values
    pred_discrete = torch.round(pred_normalized * (num_classes - 1)).long()
    label_discrete = torch.round(label_normalized * (num_classes - 1)).long()

    # Flatten the tensors if they're not already, to prepare for mIoU calculation
    pred_flat = pred_discrete.view(-1)
    label_flat = label_discrete.view(-1)

    iou_list = []
    present_iou_list = []

    for sem_class in range(num_classes):
        pred_inds = (pred_flat == sem_class)
        target_inds = (label_flat == sem_class)

        if target_inds.long().sum().item() == 0:
            iou_now = float('nan')  # Class not present in the label
        else:
            intersection_now = (pred_inds & target_inds).long().sum().item()
            union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
            iou_now = float(intersection_now) / float(union_now)
            present_iou_list.append(iou_now)

        iou_list.append(iou_now)

    # Compute the mean of the present IoU scores, ignoring NaNs
    valid_iou_scores = [iou for iou in present_iou_list if not np.isnan(iou)]
    return present_iou_list, np.mean(valid_iou_scores) if valid_iou_scores else float('nan')





In [58]:
model.eval()
val_iter = iter(val_loader)

In [63]:
import time

batch = next(val_iter)
dem = batch['DEM']
so = batch['SO']
rgbs = batch['RGB']

start = time.time()
val_outputs = model(dem, rgbs)
end = time.time()

loss = criterion(val_outputs, so)

present_iou_list, iou = mIOU(so, val_outputs)
present_iou_list_tol, valid_iou_scores, iou_tol = mIOU_with_tolerance(so, val_outputs)
present_iou_list_norm, iou_norm = normalize_and_discretize_for_mIoU(so, val_outputs)

torch.Size([1, 128, 128])
torch.Size([1, 128, 128])


/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/ipykernel_launcher.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [64]:
present_iou_list

[0.851401179941003,
 0.8007926331740296,
 0.4883808095952024,
 0.3969270166453265,
 0.18813314037626627,
 0.0,
 0.028985507246376812,
 0.0]

In [65]:
present_iou_list_tol

[0.9043030031376065,
 0.8539379474940334,
 0.8753364007292299,
 0.6728232189973615,
 0.5581283164495899,
 0.21895424836601307,
 0.04482758620689655,
 0.02702702702702703,
 0.0]

In [66]:
present_iou_list_norm

[0.851401179941003,
 0.8007926331740296,
 0.0,
 0.08716439867598381,
 0.18813314037626627,
 0.0,
 0.11494252873563218,
 0.0]